In [73]:
import re
import spacy
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [74]:
!python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 64.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [75]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
emails=pd.read_csv('/content/drive/MyDrive/spam.csv',usecols=[0, 1],encoding='ISO-8859-1')
emails.columns=['target','text']


In [77]:
emails.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [78]:
x_train,x_test,y_train,y_test=train_test_split(emails['text'],emails['target'],test_size=0.8, shuffle=True,
    random_state=42  )

In [79]:
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [80]:
nlp = spacy.load("en_core_web_md")
train_vec=np.zeros((len(x_train),300))
test_vec=np.zeros((len(x_test),300))

In [81]:
for i ,doc in tqdm(enumerate(nlp.pipe(x_train)), total=len(x_train)):
    train_vec[i, :] = doc.vector

for i ,doc in tqdm(enumerate(nlp.pipe(x_test)), total=len(x_test)):
    test_vec[i, :] = doc.vector

  0%|          | 0/1114 [00:00<?, ?it/s]

  0%|          | 0/4458 [00:00<?, ?it/s]

In [82]:
for i in random.choices(range(0, len(test_vec)), k=5):
   similarity=cosine_similarity([test_vec[i]],train_vec).flatten()
   indices = np.argsort(similarity)[::-1]
   for i, j in enumerate(indices[:3]):
        print(f"{i} nearest label is",
              f"{y_train[j]}",
              f"similarity score: {round(similarity[j], 3)}")

0 nearest label is ham similarity score: 1.0
1 nearest label is ham similarity score: 0.967
2 nearest label is ham similarity score: 0.948
0 nearest label is ham similarity score: 0.877
1 nearest label is spam similarity score: 0.875
2 nearest label is spam similarity score: 0.875
0 nearest label is ham similarity score: 0.921
1 nearest label is spam similarity score: 0.921
2 nearest label is ham similarity score: 0.921
0 nearest label is ham similarity score: 0.901
1 nearest label is ham similarity score: 0.893
2 nearest label is ham similarity score: 0.893
0 nearest label is ham similarity score: 0.968
1 nearest label is ham similarity score: 0.968
2 nearest label is ham similarity score: 0.968


In [83]:
y_predict=[]
for i in range(len(test_vec)):
   similarity=cosine_similarity([test_vec[i]],train_vec).flatten()
   indices = np.argsort(similarity)[::-1]
   nearest_label=[y_train[j] for j in indices[:3]]
   most_common_label = Counter(nearest_label).most_common(1)[0][0]
   y_predict.append(most_common_label)
accuracy = accuracy_score(y_test, y_predict)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.93
